In [1]:
%cd ../src

/Users/mikhail_zybin/food-order-chat-bot/src


In [2]:
import openai
import pandas as pd

In [3]:
from chatbot import (
    initialize_system_prompt,
    make_conversation,
    initialize_messages,
)

In [4]:
api_base = "https://llama3-1-8b-api.llm.lab.epam.com/v1"
model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
client = openai.OpenAI(api_key="***REMOVED***", base_url=api_base)

In [6]:
messages = initialize_messages()
make_conversation(messages, model, client)

Chatbot: Hello, Dear Customer!

I am food order chatbot! My function is to help you choose and order food from various cafes and restaurants.

What would you like to eat today?
You: 

 https://login.microsoftonline.com/b41b72d0-4e9f-4c26-8a69-f949f367c91d/oauth2/authorize?client%5Fid=00000003%2D0000%2D0ff1%2Dce00%2D000000000000&response%5Fmode=form%5Fpost&response%5Ftype=code%20id%5Ftoken&resource=00000003%2D0000%2D0ff1%2Dce00%2D000000000000&scope=openid&nonce=39B38370A50F1615CBE6DE7719F9FD55F2D7C8622022DE09%2D15A4FCA6FDD620E7C3FC9BB2AF09C721449D139D869CD9DCC6373575577A46A6&redirect%5Furi=https%3A%2F%2Fepam%2Dmy%2Esharepoint%2Ecom%2F%5Fforms%2Fdefault%2Easpx&state=OD0w&claims=%7B%22id%5Ftoken%22%3A%7B%22xms%5Fcc%22%3A%7B%22values%22%3A%5B%22CP1%22%5D%7D%7D%7D&wsucxt=1&cobrandid=11bd8083%2D87e0%2D41b5%2Dbb78%2D0bc43c8a8e8a&client%2Drequest%2Did=8aba6ea1%2Df04b%2Da000%2Df706%2Dc7d94f15fde4&sso_reload=true


2024-12-18 15:32:10.231 | DEBUG    | chatbot:parse_llm_json:56 - Based on the conversation between the chatbot and the user, I can see that the user has not provided any information about the restaurant they want to order from. The user's first message is a link to a Microsoft login page, which is unrelated to food ordering. The chatbot's response is a generic greeting, but it does not ask the user to specify a restaurant.

The user's second message is also unrelated to food ordering, and the chatbot does not ask the user to specify a restaurant.

Therefore, the fields of the json output must be empty lists.

```json
{
  "dish_names": [],
  "dish_quantities": [],
  "restaurant_name": "",
  "delivery_time": ""
}
```


Chatbot: I'm happy to help you order food, but it seems like you've provided a link to a Microsoft login page. I'm a chatbot, and I don't have the ability to access external links or perform tasks unrelated to food ordering.

Let's get back to ordering food! Can you please tell me what type of cuisine you're in the mood for today? Or would you like me to suggest some popular options?
You: 

KeyboardInterrupt: Interrupted by user